In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import tensorflow_probability as tfp

In [3]:
# Load MNIST
(x_dev, y_dev), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('There are', len(x_dev), 'training images.')
print('There are', len(x_test), 'test images.')

There are 60000 training images.
There are 10000 test images.


In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_dev, y_dev)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [9]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import Model


class StochasticMLP(Model):
    
    def __init__(self, hidden_layer_sizes=[100], n_outputs=10):
        super(StochasticMLP, self).__init__()
        self.hidden_layer_sizes = hidden_layer_sizes
        self.fc_layers = [Dense(layer_size) for layer_size in hidden_layer_sizes]
        self.output_layer = Dense(n_outputs)

    
    def call(self, x):
        
        x = Flatten()(x)
        
        network = []
        
        for i, layer in enumerate(self.fc_layers):
            
            logits = layer(x)
            x = tfp.distributions.Bernoulli(logits=logits).sample()
            network.append(x)     
            
        final_logits = self.output_layer(x)
        
        return network
    
    def target_log_prob(self, x, h, y):
        
        x = Flatten()(x)
        
        h_current = h
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        h_previous = [x] + h_current[:-1]
        
        log_prob = 0.
        
        for i, (cv, pv, layer) in enumerate(
            zip(h_current, h_previous, self.fc_layers)):
            
            ce = tf.nn.sigmoid_cross_entropy_with_logits( # only works for discretize version
                labels=cv, logits=layer(pv))
            log_prob += tf.reduce_sum(ce, axis = -1)
            
        log_prob += tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.cast(y, tf.int32), logits=self.output_layer(h_current[-1]))
            
        return log_prob
    
    def target_log_prob2(self, x, h_current, y):
        
        x = Flatten()(x)
        #h_current_new = [tf.split(h_current[i], [self.hidden_layer_sizes[0]]) for i in range(32)]
        h_current_new = tf.split(h_current, self.hidden_layer_sizes, axis = 1)
        h_previous = [x] + h_current_new[:-1]
        
        log_prob = 0.
        
        for i, (cv, pv, layer) in enumerate(
            zip(h_current_new, h_previous, self.fc_layers)):
            
            ce = tf.nn.sigmoid_cross_entropy_with_logits( # only works for discretize version
                labels=cv, logits=layer(pv))
            log_prob += tf.reduce_sum(ce, axis = -1)
            
        log_prob += tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.cast(y, tf.int32), logits=self.output_layer(h_current_new[-1]))
        
        #print(log_prob)
        
        return log_prob
    
    def run_chain(self, x, h, y):
        
        #h_current = h
        h_current = tf.concat([h[0], h[1]], axis=1)
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        print(np.asarray(h_current).shape)
        #print(h)
        
        def tlp(*args):
            return self.target_log_prob2(x, args, y)
        
        adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn = tlp,
            num_leapfrog_steps = 10,
            step_size = pow(1000, -1/4)),
            num_adaptation_steps=int(1000 * 0.8))
        
        # Run the chain (with burn-in).
        samples = tfp.mcmc.sample_chain(
          num_results=5,
          num_burnin_steps=1000, # set to 1000
          current_state=h_current,
          kernel=adaptive_hmc,
          trace_fn=None
          )
        
        sample_mean = tf.reduce_mean(samples)
        
        return sample_mean

In [10]:
model = StochasticMLP(hidden_layer_sizes = [100, 50], n_outputs=10)

In [7]:
network = [model.call(images) for images, labels in train_ds]

In [11]:
hmc = [model.run_chain(images, net, labels) for (images, labels), net in zip(train_ds, network)]

(32, 150)
tf.Tensor(
[5447.762  4681.638  3781.3452 3796.6902 5634.8384 5059.22   6078.3296
 3359.0115 4861.9204 5345.294  5233.9946 4466.6304 5806.1055 3008.306
 6404.3887 5488.911  3555.182  4147.587  3867.6643 4377.4795 3313.4756
 3634.813  4966.679  4763.073  4990.521  3433.9924 4251.1357 5478.0425
 4050.6519 3711.2126 6385.899  4192.5635], shape=(32,), dtype=float32)
tf.Tensor(
[5447.762  4681.638  3781.3452 3796.6902 5634.8384 5059.22   6078.3296
 3359.0115 4861.9204 5345.294  5233.9946 4466.6304 5806.1055 3008.306
 6404.3887 5488.911  3555.182  4147.587  3867.6643 4377.4795 3313.4756
 3634.813  4966.679  4763.073  4990.521  3433.9924 4251.1357 5478.0425
 4050.6519 3711.2126 6385.899  4192.5635], shape=(32,), dtype=float32)
tf.Tensor(
[35142.684 22500.828 17310.477 19624.31  35318.734 20820.828 30540.611
 10945.765 22072.906 35129.65  25654.271 21245.494 37897.246 10936.982
 40843.92  27020.012 17283.842 27527.709 24009.94  20366.26  15765.173
 22248.492 32191.744 28213.988 23324

/Users/hmy/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tensorflow_probability/python/mcmc/sample.py:341: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '


InvalidArgumentError: Inputs to operation AddN of type AddN must have the same size and shape.  Input 0: [32] != input 2: [150] [Op:AddN]